In [1]:
import matplotlib
#matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense
#from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import tensorflow as tf



from sklearn.preprocessing import LabelBinarizer
from skimage.io import imread, imshow



In [2]:
x=np.load("../data.csv.npy")
y=np.load("../labels.npy")

In [3]:
(trainX, testX, trainY, testY) = train_test_split(x, y,
test_size=0.20, stratify=y, random_state=42)

In [4]:
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
valAug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

baseModel = EfficientNetB0(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
	layer.trainable = False


headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(23, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
print("[INFO] compiling model...")
opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2/15)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=64),
	#steps_per_epoch=len(trainX) //64,
	validation_data=valAug.flow(testX, testY),
	#validation_steps=len(testX) //64,
	epochs=20)

[INFO] compiling model...
[INFO] training head...


G:\ANNACONDA\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
G:\ANNACONDA\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
195/195 [==============================] - 478s 2s/step - loss: 2.6747 - accuracy: 0.2287 - val_loss: 2.3882 - val_accuracy: 0.2960
Epoch 2/20
195/195 [==============================] - 440s 2s/step - loss: 2.4313 - accuracy: 0.2834 - val_loss: 2.2998 - val_accuracy: 0.3107
Epoch 3/20
195/195 [==============================] - 431s 2s/step - loss: 2.3308 - accuracy: 0.3057 - val_loss: 2.2150 - val_accuracy: 0.3287
Epoch 4/20
195/195 [==============================] - 439s 2s/step - loss: 2.2587 - accuracy: 0.3224 - val_loss: 2.1804 - val_accuracy: 0.3393
Epoch 5/20
195/195 [==============================] - 460s 2s/step - loss: 2.1992 - accuracy: 0.3451 - val_loss: 2.1613 - val_accuracy: 0.3525
Epoch 6/20
195/195 [==============================] - 456s 2s/step - loss: 2.1507 - accuracy: 0.3618 - val_loss: 2.1296 - val_accuracy: 0.3580
Epoch 7/20
195/195 [==============================] - 464s 2s/step - loss: 2.1174 - accuracy: 0.3665 - val_loss: 2.1214 - val_accuracy: 0.3570

In [12]:
model = keras.models.save('skinmodel')

AttributeError: module 'tensorflow.keras.models' has no attribute 'save'

In [11]:
from tensorflow import keras

G:\ANNACONDA\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: resNet_mode101l_for_fingerprint.hd5\assets


In [15]:
model.save('skinnet')

G:\ANNACONDA\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: skinnet\assets


In [17]:
model.save('skinnet.h5')

In [19]:
model.save('skinnet.hdf5')

In [21]:
model.save_weights('skinnetweights.h5')